In [1]:
import logging
import sys
from pathlib import Path

import datacube
import geopandas as gpd
import pytest

import deafrica_conflux.drill
from deafrica_conflux.id_field import guess_id_field
from deafrica_conflux.plugins.utils import validate_plugin, run_plugin

logging.basicConfig(level=logging.INFO)

In [2]:
# Connect to datacube.
dc = datacube.Datacube(app='test')

In [3]:
plugin = run_plugin("plugins/sum_wet.py")
uuid = "6d1d62de-5edd-5892-9dcc-9e1616251411"
datasets = deafrica_conflux.drill.find_datasets_for_plugin(dc, plugin, uuid)
assert len(datasets) == 1
assert str(datasets["wofs_ls"].id) == uuid

In [4]:
plugin = run_plugin("plugins/timeseries.py")
uuid = "d15407ff-3fe5-55ec-a713-d4cc9399e6b3"
polygons_gdf = gpd.read_file("data/edumesbb2.geojson")

id_field = guess_id_field(polygons_gdf, "UID")
polygons_gdf.set_index(id_field, inplace=True)

drill_result = deafrica_conflux.drill.drill(plugin,
                                            polygons_gdf,
                                            uuid,
                                            partial=True,
                                            dc=dc)
assert len(drill_result) == pytest.approx(86, 1)
# 7 columns, 3 output and 4 directions
assert len(drill_result.columns) == 7
assert "conflux_n" in drill_result.columns

INFO:deafrica_conflux.drill:Reference scene is Frozen({'time': 1, 'y': 8556, 'x': 6722})


In [5]:
uid = "effd8637-1cd0-585b-84b4-b739e8626544"
polygons_gdf = gpd.read_file("data/edumesbb2.geojson")

ds_extent = dc.index.datasets.get(uid).extent.to_crs(polygons_gdf.crs)
ds_extent_geom = ds_extent.geom

intersection = polygons_gdf.geometry.intersection(ds_extent_geom)
dirs = deafrica_conflux.drill._get_directions(polygons_gdf.geometry[0], intersection.geometry[0])
assert dirs == {"South"}

In [17]:
plugin = run_plugin("plugins/timeseries.py")
uuid = "effd8637-1cd0-585b-84b4-b739e8626544"
polygons_gdf = gpd.read_file("data/edumesbb2.geojson")

id_field = guess_id_field(polygons_gdf, "UID")
polygons_gdf.set_index(id_field, inplace=True)

drill_result = deafrica_conflux.drill.drill(plugin,
                                            polygons_gdf,
                                            uuid,
                                            partial=True,
                                            overedge=True,
                                            dc=dc)
assert len(drill_result) == 1
assert drill_result["wet_percentage"].iloc[0] == 42.39275304214028

INFO:deafrica_conflux.drill:Reference scene is Frozen({'time': 2, 'y': 507, 'x': 443})


In [21]:
from deafrica_conflux.hopper import find_datasets
datasets = find_datasets(query={}, products=["wofs_ls"], dc=dc)
dss = list(datasets)
for ds in dss:
    uuid = ds.id
    #polygons_gdf = gpd.read_file("data/edumesbb2.geojson")
    #polygons_gdf = gpd.read_file("data/edvntx0mu.geojson")
    polygons_gdf = gpd.read_file("data/edumesbb2.geojson")
    
    ds_extent = dc.index.datasets.get(uid).extent.to_crs(polygons_gdf.crs)
    ds_extent_geom = ds_extent.geom
    
    intersection = polygons_gdf.geometry.intersection(ds_extent_geom)
    dirs = deafrica_conflux.drill._get_directions(polygons_gdf.geometry[0], intersection.geometry[0])
    print(dirs)

{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}
{'South'}


In [ ]:
wet_percentage